# Textklassifikation

## Datensatz laden

In [ ]:
from sklearn import datasets

In [ ]:
data = datasets.fetch_20newsgroups(shuffle=True)

## Bag of Words

**Aufgabe 1:** Verwende zunächst die ersten 10 Dokumente aus dem Trainings-Datensatz, um Schritt für Schritt eine Funktion `tokenize_documents` aufzubauen.

1) Entferne alle Zeilenumbrüche (`"\n"`) und konvertiere jedes Dokument in Kleinbuchstaben.
2) Ersetze alle Satzzeichen usw. durch Leerzeichen.
3) Splitte jedes Dokument in einzelne Wörter. (Achte darauf, dass die Liste der Wörter keine leeren Einträge `""` enthält.)

In [ ]:
docs = x_train[:10]

In [ ]:
def tokenize_documents(docs):
    tokenized = []
    for doc in docs:
        # START Dein Code
        
        ...
        
        # ENDE
        tokenized.append(doc)
    
    return docs

tokenize_documents(docs)

**Aufgabe 2:**

Welche Wörter kommen im Datensatz vor? Wie oft kommen diese vor?

Erstelle dazu ein Dictionary `vocab` dessen Schlüssel die Wörter und dessen Wert jeweils die Anzahl des Vorkommens des Worts ist.

In [ ]:
tokenized_docs = tokenize_documents(x_train[:10])

In [ ]:
def create_vocabulary(tokenized_docs):
    vocab = {}
    
    for doc in tokenized_docs:
        # START Dein Code

        ...
    
        # ENDE
    
    return vocab

create_vocabulary(tokenized_docs)

**Aufgabe 3:* Erstelle ein Vokabular für den gesamten Trainings-Datensatz.

Wie sind die Wort-Anzahlen verteilt? Plotte z.B. ein Histogramm.

In [ ]:
vocab = create_vocabulary(tokenize_documents(x_train))

In [ ]:
**Aufgabe 4:** Was sind die häufigsten Wörter? In wie viel Prozent der Dokumente kommen diese vor? Sind diese Wörter geeignet, um die verschiedenen Kategorien voneinander zu unterscheiden?

**Aufgabe 5:** Schreibe eine Funktion, die die häufigsten Wörter ohne Unterscheidungskraft (sog. "stopwords") aus den Dokumenten entfernt.

In [ ]:
tokenized_docs = tokenize_documents(x_train[:10])

stopwords = ...

def remove_stopwords(tokenized_docs):
    cleaned_docs = []
    for doc in tokenized_docs:
        # START Dein Code

        ...
    
        # ENDE
        cleaned_docs.append(doc)
    
    return cleaned_docs

remove_stopwords(tokenized_docs)

**Aufgabe 6:** Erstelle das Vokabular des gesamten Trainings-Datensatzes ohne Stopwords.

**Aufgabe 7:** Erstelle eine Liste `feature_words` der 1.000 häufigsten Wörter.

Erstelle eine Funktion `extract_features(preprocessed_documents, feature_words)` die jedes Dokument eine Liste `bow_doc` der Länge 1000 konvertiert.

Jeder Eintrag der Liste entspricht einem Wort aus `feature_words`. Wenn das Wort in dem Dokument vorkommt, ist der Eintrag `1`, ansonsten `0`.

Zum Beispiel: Wenn `feature_words[5]` das Wort "Hund" ist und "Hund" im Dokument vorkommt, dann ist `bow_doc[5] == 1`.

In [ ]:
feature_words = ...

tokenized_docs = remove_stopwords(tokenize_documents(x_train[:10]))

def extract_features(tokenized_docs, feature_words):
    ...

extract_features(tokenized_docs, feature_words)

**Aufgabe 8:** Kombiniere die erstellten Funktionen zu einem kompletten Text-Preprocessing:

- Test- und Trainingsdaten mit `tokenize_documents` und `remove_stopwords` vorbereiten.
- Aus den Trainingsdaten das Vokabular ermitteln (aus Aufgabe 6 kopieren).
- Aus dem Vokabular die `feature_words` ermitteln.
- Test- und Trainingsdaten mit `extract_features` in Vektoren umwandeln.

## CountVectorizer

## Klassifikation mit Naive Bayes

**Idee:** 

Für jedes Wort $x_i$ gibt es eine Wahrscheinlichkeit, dass es in einem Text der Kategorie $y$ auftaucht: $P(x_i | y )$

Die Wahrscheinlichkeit, dass ein Text zur Kategorie $y$ gehört, ist dann $P(y | x_1, ..., x_n) = P(x_1 | y) P(x_2 | y) ... = \prod_i^n P(x_i | y)$

In unserem Fall nehmen wir an, dass $P(x_i|y) = \frac{N_{y,i} + \alpha}{N_y + \alpha n}$, wobei $N_{y, i}$ die Anzahl ist, mit der $x_i$ in Texten der Kategorie $y$ auftaucht und $N_y$ die Gesamtzahl aller Worte/Features in der Klasse $y$ (also die Summe aller $N_{y,i}$).



## Term Frequency - Inverse Document Frequency (tf-idf)

**Term Frequency:** Anzahl der Vorkommen eines Worts gewichtet nach der Länge des Texts.

**Document Frequency:** Worte die in vielen Texten vorkommen werden geringer gewichtet als Worte, die nur in wenigen Texten vorkommen.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.utils._param_validation import InvalidParameterError

tr = []
ts = []
ps = []
par = "max_df"
for i in range(1, 100):
    p = 1 - np.log10(i/10)

    tf_idf = TfidfVectorizer(max_features=100, **{par: p})
    
    try:
        _x = tf_idf.fit_transform(x_train)
    except InvalidParameterError:
        continue
    _xt = tf_idf.transform(x_test)
    
    _m = MultinomialNB()
    _m.fit(_x, y_train)
    ps.append(p)
    tr.append(_m.score(_x, y_train))
    ts.append(_m.score(_xt, y_test))
    
plt.plot(ps, tr, label="training")
plt.plot(ps, ts, label="test")
plt.xlabel(par)
plt.ylabel("score")
plt.legend()